In [ ]:
# # import library
# import cv2
# import matplotlib.pyplot as plt
# import numpy as np
# import os
# import pickle

# from PIL import Image

# headshots_folder_path = "data/headshots"

# # image dimensions
# image_width = 224
# image_height = 224

# # for detecting faces
# facecascade = cv2.CascadeClassifier('cascade_frontalface.xml')

# # set the directory containing the images
# images_dir = os.path.join("..", headshots_folder_path)

# cur_id = 0
# label_ids = {}

# # iterates through all the files in each subdirectories
# for root, _, files in os.walk(images_dir):
#     for file in files:
#         if file.endswith('png') or\
#         file.endswith('jpg') or\
#         file.endswith('jpeg'):
#             # path of the  image
#             path = os.path.join(root, file)

#         # get the label name (name of the person)
#         label = os.path.basename(root).replace(" ", "..").lower()

#         # add the label in label_ids:
#         label_ids[label] = cur_id
#         cur_id += 1

#         # load the image
#         imgtest = cv2.imread(path, cv2.IMREAD_COLOR)
#         image_array = np.array(imgtest, "uint8")

#         # get the faces detected in the image
#         faces = facecascade.detectMultiScale(imgtest, scaleFactor=1.1, minNeighbors=5)

#         # if not exactly 1 face is detected, skip this photo
#         if len(faces) != 1:
#             print(f'--Photo Skipped--\n')

#         # remove the original image
#         os.remove(path)
#         continue

#         for (x_, y_, w, h) in faces:

#             # draw the face detected
#             face_detect = cv2.rectangle(imgtest,
#                                         (x_, y_),
#                                         (x_+w, y_+h),
#                                         (255, 0, 255), 2)
#             plt.imshow(face_detect)
#             plt.show()

#             # resize the detected face to 224x224
#             size = (image_width, image_height)

#             # detected face region
#             roi = image_array[y_: y_ + h, x_: x_ + w]

#             # resize the detected head to target size
#             resized_image = cv2.resize(roi, size)
#             image_array = np.array(resized_image, "uint8")

#             # remove the original image
#             os.remove(path)

#             # replace the image with only the face
#             im = Image.fromarray(image_array)
#             im.save(path)


In [16]:
# import libraries
import cv2
from cv2 import CascadeClassifier, imread

# check cv2 version
print("OpenCV version: " + cv2.__version__)

OpenCV version: 4.7.0


## How to Detect Faces for Face Recognition
Before we can perform face recognition, we need to detect faces.

`Face detection` is the process of automatically locating faces in a photograph and localizing them by drawing a bounding box around their extent.

In [26]:
# import libraries
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN
from numpy import asarray
from PIL import Image

In [27]:
# load image from file
img = plt.imread("headshots/Marwan Musa/1.jpg")

# create the detector, using default weights
detector = MTCNN()

# detect faces in the image
res = detector.detect_faces(img)

1/1 [==============================] - 0s 107ms/step


In [31]:
# extract the bounding box from the first face
x1, y1, width, height = res[0]['box']
x2, y2 = x1+width, y1+height

# extract the face
face = img[y1:y2, x1:x2]

In [33]:
# resize pixels to the model size
image = Image.fromarray(face)
image = image.resize((224, 224))
face_array = asarray(image)

In [34]:
type(face_array)

numpy.ndarray

In [4]:
# unite all the code above in a function extract_face()

# import libraries
import matplotlib.pyplot as plt
import numpy

from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from mtcnn.mtcnn import MTCNN
from numpy import asarray
from PIL import Image
from scipy.spatial.distance import cosine

# extract a single face from a given photograph
def extract_face(filename : str, required_size : tuple = (224, 224)) -> numpy.array:
    # load img from file
    img = plt.imread(filename)

    # create detector using default weigths
    detector = MTCNN()

    # detect faces in the image
    res = detector.detect_faces(img)

    # extract the bounding box from the first face
    x1, y1, width, height = res[0]['box']
    x2, y2 = x1+width, y1+height

    # extract the face
    face = img[y1:y2, x1:x2]

    # resize image to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

In [ ]:
# load the photo and extract the face
img = extract_face("headshots/Kevin Hart/3.jpg")
# plot the extracted face
plt.imshow(img)
# show the plot
plt.show()

In [5]:
# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
    #extract faces
    faces = [extract_face(filename) for filename in filenames]
    # convert into an array of samples
    samples = asarray(faces, 'float32')
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    # create a vggface model
    model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    # perform prediction
    yhat = model.predict(samples)
    return yhat

# determine if a candidate face is a match for a know face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
    # calculate distance between embeddings
    score = cosine(known_embedding, candidate_embedding)
    if score <= thresh:
        print(">face is a Match (%.3f <= %.3f)" % (score, thresh))
    else:
        print(">face is NOT a Match (%.3f > %.3f)" % (score, thresh))


In [13]:
# define filenames
filenames = [
    "headshots/Marwan Musa/1.jpg",
    "headshots/Marwan Musa/2.jpg",
    "headshots/Marwan Musa/3.jpg",
    "headshots/Kevin Hart/1.jpg"
]

In [14]:
# get embeddings file filenames
embeddings = get_embeddings(filenames)
# define my face
myface_id = embeddings[0]
# verify known photos of me
print('Positive Tests')
is_match(myface_id, embeddings[1])
is_match(myface_id, embeddings[2])
# verify known photos of other people
print('Negative Tests')
is_match(myface_id, embeddings[3])

1/1 [==============================] - 1s 845ms/step
Positive Tests
>face is a Match (0.290 <= 0.500)
>face is a Match (0.252 <= 0.500)
Negative Tests
>face is NOT a Match (0.791 > 0.500)
